In [ ]:
import pandas as pd
import spacy
import steamreviews
import json
import csv
import os
import glob
import json
import matplotlib.pyplot as plt
import nltk
import wordcloud
import re
import plotly
import seaborn as sns
import stanza
import string 
import joblib

from scipy.stats import pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from wordcloud import WordCloud
from pandas.io.json import json_normalize
from nltk.corpus import stopwords
from tqdm.notebook import trange, tqdm
from sklearn.cluster import KMeans
from collections import defaultdict
from scipy.stats import pearsonr
from collections import Counter
import matplotlib as mpl

## Reviews download
https://pypi.org/project/steamreviews/

In [ ]:
dir_name = "data"
reviews_jsons = glob.glob(f"{dir_name}/*")
reviews_jsons

In [ ]:
title_dict = {"219150": "Hotline Miami",
              "1172620" : "Sea of Thieves",
              "460950" : "Katana ZERO",
              "262060" : "Darkest Dungeon",
              "250320" : "The Wolf Among Us",
              "210970" : "The Witness",
              "620" : "Portal 2",
              "736260" : "Baba Is You",
              "219890" : "Antichamber",
              "289070" : "Sid Meier's Civilization VI",
              "427520" : "Factorio",
              "257510" : "The Talos Principle",
             "304430" : "INSIDE",
             "400" : "Portal",
             "394360" : "Hearts of Iron IV",
             "632470" : "Disco Elysium",
             "294100" : "RimWorld",
              "646570" : "Slay the Spire",
              "1150690" : "Omori",
              "1341290" : "We Were Here Forever",
              "1515210" : "The Past Within",
              "860510" : "Little Nightmares II",
              "550" : "Left 4 Dead 2"}

In [ ]:
genre_dict = {"219150": "A",
              "1172620" : "A",
              "460950" : "A",
              "262060" : "R",
              "250320" : "R",
              "210970" : "P",
              "620" : "P",
              "736260" : "P",
              "219890" : "P",
              "289070" : "S",
              "427520" : "S",
              "257510" : "P",
              "304430" : "P",
             "400" : "P",
             "394360" : "S",
             "632470" : "R",
             "294100" : "S",
             "646570" : "R",
             "1150690" : "R",
             "1341290" : "P",
             "1515210" : "P",
             "860510" : "P",
             "550" : "A"}

In [ ]:
data_list = []
for i in reviews_jsons:
    with open(i, encoding = "utf-8") as json_file:
        json_data = json.load(json_file)
        reviews = json_data['reviews']
        file_name = json_file.name.split('.')[0].split('_')[1]
        for key in reviews.keys():
            data_list.append([title_dict.get(file_name),genre_dict.get(file_name),reviews[key]['author']['steamid'], reviews[key]['votes_up'], 
                          reviews[key]['review'], reviews[key]['weighted_vote_score'], reviews[key]['voted_up'],
                          reviews[key]['author']['num_games_owned'], reviews[key]['author']['playtime_at_review'],
                          reviews[key]['author']['num_reviews'], 
                          reviews[key]["timestamp_created"] ])
df = pd.DataFrame(data_list, columns = ["title", "genre","steamid", "votes_up", "review", "weighted_vote_score", "voted_up",
                                        "num_games_owned", "playtime_at_review", "num_reviews", "timestamp_created"])

In [ ]:
df.sample(frac=0.3).head()

In [ ]:
df['timestamp_created'] = pd.to_datetime(df['timestamp_created'], unit='s')

## Preprocessing

1. Braki danych
2. Usunięcie duplikatów
4. Usunięcie recenzjii krótszych niż 3 słowa
9. Usunięcie znaków spcjalnych 
10. "Text lowering"
6. Usunięcie znaków interpunkcyjnych i liczb
8. Usunięcie tytułów gier z recenzji
7. Tokenizacja
8. Part-of-speech tagging
9. Lematyzacja

#### Braki danych

In [ ]:
df.isna().sum()

#### Usunięcie duplikatów

In [ ]:
df.duplicated(subset='review').sum()

In [ ]:
df[df.duplicated(subset='review')].sample(20)

In [ ]:
(df.duplicated(subset='review') == False).sum()

In [ ]:
df = df[df.duplicated(subset='review') == False]

In [ ]:
df.duplicated(subset='review').sum()

#### Długość recenzji

In [ ]:
df["temp_review_len"] = df["review"].apply(lambda x: x.split(" ").__len__())
df["temp_review_len"].describe()

In [ ]:
df[df["temp_review_len"] < 4].groupby(by=["genre"]).count()

In [ ]:
df = df[df["temp_review_len"] >= 3]

In [ ]:
suma = 0
for i in df["temp_review_len"]:
    suma += i

In [ ]:
print("Ilość słów:", suma,
     "Ilość recenzji:", len(df["temp_review_len"]))

In [ ]:
df = df.drop(["temp_review_len"], axis=1)

### Wordcloud

In [ ]:
txt = ' '.join(i for i in df["review"])
wordcloud = WordCloud(max_font_size = 180,
                max_words = 120,
                width = 1200,
                height = 800,
                background_color = "black",
                colormap='RdPu_r').generate(txt)

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
txt = ' '.join(i for i in df[df['genre'] == "A"]['review'])
wordcloud = WordCloud(max_font_size = 150,
                max_words = 30,
                width = 600,
                height = 400,
                background_color = "black",
                colormap='RdPu_r').generate(txt)

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
txt = ' '.join(i for i in df[df['genre'] == "R"]['review'])
wordcloud = WordCloud(max_font_size = 150,
                max_words = 30,
                width = 600,
                height = 400,
                background_color = "black",
                colormap='RdPu_r').generate(txt)

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
txt = ' '.join(i for i in df[df['genre'] == "P"]['review'])
wordcloud = WordCloud(max_font_size = 150,
                max_words = 30,
                width = 600,
                height = 400,
                background_color = "black",
                colormap='RdPu_r').generate(txt)

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
txt = ' '.join(i for i in df[df['genre'] == "S"]['review'])
wordcloud = WordCloud(max_font_size = 150,
                max_words = 30,
                width = 600,
                height = 400,
                background_color = "black",
                colormap='RdPu_r').generate(txt)

plt.figure(figsize=(12,6))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

#### Usunięcie znaków, text lowering

In [ ]:
df["review"] = df["review"].str.replace(r"\[.*?\]|\n|[^\x00-\x7F]+", "", regex=True).str.lower()

In [ ]:
df["review"].sample(10)

#### Usunięcie znaków interpunkcyjnych i liczb

In [ ]:
df["review"] = df["review"].str.replace(r"[0-9]", "", regex=True)

In [ ]:
df["review"] = df["review"].str.replace('['  + string.punctuation + ']', "", regex =True)

In [ ]:
df["review"].sample(5)

#### Usunięcie tytułów gier z recenzji

In [ ]:
for value in title_dict.values():
    df["review"] = df["review"].str.replace(value.lower(), "")

#### Tokenizacja

In [ ]:
df["tokenized"] = df["review"].apply(nltk.word_tokenize)

In [ ]:
df["tokenized"].sample(15)

### Stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
english_stopwords = stopwords.words('english')
english_stopwords_cleaned = [word.replace("'", "") for word in english_stopwords]

In [ ]:
new_column = []
for row in df["tokenized"]:
    new_row = [word for word in row if word[0] not in english_stopwords_cleaned]
    new_column.append(new_row)
df["final_review"] = new_column

## Describe

In [ ]:
df.dtypes

In [ ]:
df["weighted_vote_score"] = pd.to_numeric(df["weighted_vote_score"])

In [ ]:
df["voted_up"] = pd.to_numeric(df["voted_up"])

In [ ]:
df.describe()

In [ ]:
corr = df.corr(method = "pearson")
plt.figure(figsize=(7,5))
sns.heatmap(corr, cmap=sns.cubehelix_palette(as_cmap=True),annot=True)
plt.show()

## Wykresy

In [ ]:
value_counts_title = df["title"].value_counts()
genre_colors = {"A": sns.cubehelix_palette(4)[1], "R": sns.cubehelix_palette(4)[2], "P": sns.cubehelix_palette(4)[3], 
                "S": sns.cubehelix_palette(4)[0]}
df['color'] = df['genre'].map(genre_colors)
colors = [df.loc[df['title'] == title]['color'].iloc[0] for title in value_counts_title.index]

plt.figure(figsize=(12, 6))
x = value_counts_title.plot(kind = "bar", color = colors)
plt.ylabel("Ilość recenzji")
plt.xticks(rotation = 90)
for i, count in enumerate(value_counts_title):
    x.text(i, count, count, va = "bottom", ha = "center")

handles = [plt.Rectangle((0,0), 1,1, color=genre_colors[label]) for label in genre_colors]
plt.legend(handles, genre_colors.keys(), title="Gatunek", loc="upper right")


plt.show()
df = df.drop(["color"], axis=1)

In [ ]:
value_counts_genre = df["genre"].value_counts()
colors = sns.cubehelix_palette(4)
x = plt.pie(value_counts_genre, labels = value_counts_genre.index, colors=colors, autopct='%1.2f%%')
plt.show()

In [ ]:
value_counts_recommendation_genre = df.groupby('genre').voted_up.value_counts().unstack(0)
percentage_pos = value_counts_recommendation_genre.iloc[1] / (value_counts_recommendation_genre.iloc[1] + value_counts_recommendation_genre.iloc[0]) * 100 
percentage_neg = value_counts_recommendation_genre.iloc[0] / (value_counts_recommendation_genre.iloc[0] + value_counts_recommendation_genre.iloc[1]) * 100 

total_count = percentage_pos + percentage_neg

def addlabels_neg(x,y):
    for i in range(len(y)):
        plt.text(i, x[i] + 2, f"{x[i]:.2f}%",  ha = "center")

def addlabels_pos(x,y):
    for i in range(len(y)):
        plt.text(i, x[i] - 6, f"{y[i]:.2f}%",  ha = "center")

fig, ax1 = plt.subplots(figsize=(7, 4))
ax2 = ax1.twinx()

total_count.plot(kind="bar", color="#95DBE5FF",  label="Recommended")
percentage_neg.plot(kind="bar", color="#078282FF", label="Not Recommended")

ax1.set_xlabel("Gatunki")
ax1.tick_params(axis="y", left=False, labelleft=False)
ax2.tick_params(axis="y", right=False, labelright=False)

addlabels_neg(percentage_neg, total_count) 
addlabels_pos(total_count, percentage_pos) 

plt.legend(bbox_to_anchor=(0.85, 1.15), ncol=2)
plt.show()

In [ ]:
import numpy as np

value_counts_recommendation_genre = df.groupby('genre').voted_up.value_counts().unstack(0)
percentage_pos = value_counts_recommendation_genre.iloc[1] / (value_counts_recommendation_genre.iloc[1] + value_counts_recommendation_genre.iloc[0]) * 100 
percentage_neg = value_counts_recommendation_genre.iloc[0] / (value_counts_recommendation_genre.iloc[0] + value_counts_recommendation_genre.iloc[1]) * 100 

total_count = percentage_pos + percentage_neg

def addlabels_neg(x,y):
    for i in range(len(y)):
        plt.text(i, x[i] + 2, f"{x[i]:.2f}%",  ha = "center")

def addlabels_pos(x,y):
    for i in range(len(y)):
        plt.text(i, x[i] - 6, f"{y[i]:.2f}%",  ha = "center")

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

total_count.plot(kind="bar", color="#ffdaf6",  label="Recommended")
percentage_neg.plot(kind="bar", color="#b34a98", label="Not Recommended")

axs[1].set_xlabel("Gatunki")
addlabels_neg(percentage_neg, total_count) 
addlabels_pos(total_count, percentage_pos) 
axs[1].legend(loc=(1.04, 0))

value_counts_genre = df["genre"].value_counts()
colors = sns.cubehelix_palette(6)

axs[0].pie(value_counts_genre, labels = value_counts_genre.index,colors=colors, autopct='%1.1f%%', startangle=140)
plt.show()

## Dependency parser test

In [ ]:
test = [df["final_review"][6543], df["final_review"][4534], df["final_review"][12345], df["final_review"][12354],
        df["final_review"][1925], df["final_review"][1067],df["final_review"][22951],df["final_review"][2231],
       ["not","fun", "gameplay"], df["tokenized"][832], df["final_review"][21], df["tokenized"][27061],
       df["final_review"][11088], df["final_review"][3171], df["final_review"][7834], df["final_review"][20180],
       df["final_review"][6697], df["final_review"][893]]

In [ ]:
import stanza
nlp = stanza.Pipeline('en', tokenize_pretokenized=True) 
doc = nlp(test)

In [ ]:
dependency_list = []
for dependency in doc.sentences[13].dependencies:
    dependency_list.append([dependency[2].text, dependency[0].id, dependency[1]])
    
for i in range(0, len(dependency_list)):
    if (int(dependency_list[i][1]) != 0):
        dependency_list[i][1] = test[13][(int(dependency_list[i][1]) - 1)]

print(*dependency_list[:10], sep='\n')

In [ ]:
feature_list = []
feature_opinion_list = [] 

for dependency in doc.sentences[13].dependencies:
    if dependency[2].xpos == "RB":
        opinion_word = dependency[2]
        rb_adj = dependency[0]
        
    if (dependency[2].xpos in ["NN", "NNS","JJ", "JJS", "JJR"]) and (dependency[1] in ["amod"]):
        if dependency[0] == rb_adj or dependency[2] == rb_adj:
            if dependency[2].xpos in ["JJS", "JJR"]:
                result_dict = {'feature': dependency[0].lemma, 'opinion': opinion_word.text + " " + dependency[2].text}
            else:
                result_dict = {'feature': dependency[0].lemma, 'opinion': opinion_word.text + " " + dependency[2].lemma}     
        else:
            if dependency[2].xpos in ["JJS", "JJR"]:
                result_dict = {'feature': dependency[0].lemma, 'opinion': dependency[2].text}
            else:
                result_dict = {'feature': dependency[0].lemma, 'opinion': dependency[2].lemma}  

        feature_list.append(dependency[0].lemma)
        feature_opinion_list.append(result_dict)

print(f'Feature list: {feature_list}')
print(f'Feature-opinion list: {feature_opinion_list}')

## Dependency parser final

In [ ]:
with open('opinions.json', 'r') as file:
    dct = json.load(file)

In [ ]:
feature_list = []

for i in range(len(df)):
    index = str(i)  
    feature = dct.get(index)
    if feature is None:
        feature_list.append([])
    else:
        feature_list.append(feature)

df["feature"] = feature_list

In [ ]:
words = []
for i in df["feature"]:
    for j in i:
        words.append(j["feature"])

In [ ]:
unique_words = list(set(words))

In [ ]:
word_counts = pd.DataFrame([[i, len([q for q in words if q==i])] for i in unique_words], 
              columns=['word', 'count']).sort_values('count', ascending=False)

## Dendrogram

In [ ]:
popular_words = word_counts[word_counts['count']>= 15]["word"].to_list()
nlp = spacy.load('en_core_web_md')
tokens = [nlp(word) for word in popular_words]
word_vectors = pd.DataFrame([i.vector for i in tokens])

In [ ]:
Z = linkage(word_vectors.values, method='ward')

plt.figure(figsize=(10,6))
dendrogram(Z, p = 5, truncate_mode = "level", orientation='top', labels = popular_words, leaf_font_size = 8)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Words')
plt.ylabel('Distance')
plt.show()

In [ ]:
cluster_labels = fcluster(Z, t = 75,  criterion="distance")

clusters = {}
for word, label in zip(popular_words, cluster_labels):
    if label not in clusters:
        clusters[label] = [word]
    else:
        clusters[label].append(word)

for label, words in clusters.items():
    print(f"Cluster {label}: {', '.join(words)}")

In [ ]:
def words_to_list(words_string):
    return words_string.split()

clusters_df = pd.read_csv('clusters.csv', converters={'Words': words_to_list})
clusters_df

In [ ]:
clusters = {
    7: 'Walka',
    95: 'Walka',
    11: 'Walka',
    75: 'Rozgrywka',
    19: 'Mechanika gry',
    99: 'Mechanika gry',
    25: 'Wyzwania',
    76: 'Grafika',
    87: 'Grafika',
    67: 'Grafika',
    56: 'Humor',
    62: 'Narracja i fabuła',
    22: 'Narracja i fabuła',
    63: 'Błędy i usterki',
    77: 'Aktualizacje i poprawki',
    21: 'Społeczność i współpraca',
    12: 'Koszt',
    24: 'Koszt',
    18: 'Jakość treści',
    8: 'Misje i przygody',
    16: 'Produkcja i rozwój',
    72: 'Opinie użytkowników',
    48: 'Satysfakcja'
}

In [ ]:
categories = clusters_df.groupby(clusters_df['Cluster'].map(clusters)).agg({'Words': 'sum'}).reset_index()
categories

In [ ]:
categories["Words"][15]

## Similarity

In [ ]:
words = word_counts["word"].to_list()
nlp = spacy.load('en_core_web_md')
tokens = [nlp(word) for word in words]

In [ ]:
categories['WordTokens'] = categories['Words'].apply(lambda words: [nlp(word) for word in words])
all_cluster_words = set(word for words in categories['Words'] for word in words)

for token in tokens:
    max_similarity = 0
    best_cluster = None
    
    for idx, row in categories.iterrows():
        for word_token in row['WordTokens']:
            similarity = token.similarity(word_token)
            if similarity > max_similarity:
                max_similarity = similarity
                best_cluster = idx
    
    if max_similarity > 0.75 and str(token) not in all_cluster_words:
        categories.at[best_cluster, 'Words'].append(str(token))
        all_cluster_words.add(token)

categories.drop(columns=['WordTokens'], inplace=True)
print(categories)

In [ ]:
categories["Words"][15]

# Feature-opinion-category list

In [ ]:
for i in df["feature"]:
    for j in i:
        for index, row in categories.iterrows():
            if j["feature"] in row["Words"]:
                j["category"] = row["Cluster"]

# Sentiment

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

In [ ]:
for i in df["feature"]:
    for j in i:
        expr = j["opinion"] #+ " " + j["feature"] 
        scores = analyzer.polarity_scores(expr)
        j["sentiment"] = scores["compound"]

### Sentiment for each feature category for different genres

In [ ]:
category_genre_sentiment_count = {}

for genre, features in zip(df["genre"], df["feature"]):

    if genre not in category_genre_sentiment_count:
        category_genre_sentiment_count[genre] = defaultdict(lambda: {"positive": 0, "negative": 0, "neutral": 0})
    
    for feature in features:
        if "category" in feature and "sentiment" in feature:
            category = feature["category"]
            sentiment = feature["sentiment"]
            
            if category not in category_genre_sentiment_count[genre]:
                category_genre_sentiment_count[genre][category] = {"positive": 0, "negative": 0, "neutral": 0}
            
            if sentiment > 0:
                category_genre_sentiment_count[genre][category]["positive"] += 1
            elif sentiment < 0:
                category_genre_sentiment_count[genre][category]["negative"] += 1
            else:
                category_genre_sentiment_count[genre][category]["neutral"] += 1
                
    for category in category_genre_sentiment_count[genre]:
        total = sum(category_genre_sentiment_count[genre][category].values())
        for sentiment in category_genre_sentiment_count[genre][category]:
            category_genre_sentiment_count[genre][category][sentiment] = \
                (category_genre_sentiment_count[genre][category][sentiment] / total) * 100
    
    category_genre_sentiment_count[genre] = dict(category_genre_sentiment_count[genre])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle("Proporcje nastrojów w poszczególnych kategoriach", fontsize=16)

def plot_genre_sentiments(ax, genre, title):
    categories = list(category_genre_sentiment_count[genre].keys())
    positive = np.array([category_genre_sentiment_count[genre][cat]["positive"] for cat in categories])
    negative = np.array([category_genre_sentiment_count[genre][cat]["negative"] for cat in categories])
    neutral = np.array([category_genre_sentiment_count[genre][cat]["neutral"] for cat in categories])

    ax.bar(categories, positive, color="#96d800", label="Positive", width=0.5)
    ax.bar(categories, neutral, bottom=positive, color="#fce655", label="Neutral", width=0.5)
    ax.bar(categories, negative, bottom=neutral + positive, color="#b20918", label="Negative", width=0.5)

    ax.tick_params(axis="x", rotation=90)
    ax.set_title(title, fontsize=12)
    ax.grid(True, which='major', axis='y', linestyle='--', alpha=0.5)

plot_genre_sentiments(axs[0, 0], "R", "RPG")
plot_genre_sentiments(axs[0, 1], "A", "Akcja")
plot_genre_sentiments(axs[1, 0], "P", "Puzzle")
plot_genre_sentiments(axs[1, 1], "S", "Strategia")

plt.tight_layout(rect=[0, 0, 1, 0.95])

fig.legend(["Pozytywne", "Neutralne", "Negatywne"], loc="upper center", bbox_to_anchor=(0.5, 0.95), ncol=3, fontsize=11)

plt.show()

### Feature category significance analysis for different genres

In [ ]:
genre_category = {}
for genre, features in zip(df["genre"], df["feature"]):
    if genre not in genre_category:
        genre_category[genre] = []
    for feature in features:
        if "category" in feature:
            genre_category[genre].append(feature["category"])

In [ ]:
genre_category_counts = {genre: pd.Series(categories).value_counts() for genre, categories in genre_category.items()}

In [ ]:
genre_category_counts = pd.DataFrame(genre_category_counts)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 11.5))
fig.suptitle("Liczba cech według gatunku z podziałem na kategorie", fontsize=16)

values = genre_category_counts["R"].sort_values(ascending=False)[1:].keys()
cmap = mpl.colormaps["turbo"]
colors = cmap(np.linspace(0, 1, len(values)))
value_color_dict = dict(zip(values, colors))

def plot_genre_category_count(ax, genre, title):
    categories = list(genre_category_counts[genre].sort_values(ascending=False)[1:].keys())
    counts = list(genre_category_counts[genre].sort_values(ascending=False)[1:])
    bars = ax.bar(categories, counts, color=[value_color_dict[category] for category in categories])
    ax.set_title(title, fontsize=12)
    ax.set_ylabel("Liczba", fontsize=10)
    ax.tick_params(axis="x", rotation= 90, labelsize=10)
    ax.grid(True, which='major', axis='y', linestyle='-', alpha=0.5)
    
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval +0.5 , round(yval, 1), ha='center', va='bottom', fontsize=8)

plot_genre_category_count(axs[0, 0], "R", "RPG")
plot_genre_category_count(axs[0, 1], "A", "Akcja")
plot_genre_category_count(axs[1, 0], "P", "Puzzle")
plot_genre_category_count(axs[1, 1], "S", "Strategia")

plt.tight_layout(rect=[0, 0, 1, 0.99])

plt.show()

In [ ]:
def pie_data(counts, categories, top_n=6):
    sorted_indices = sorted(range(len(counts)), key=lambda i: counts[i], reverse=True)
    sorted_counts = [counts[i] for i in sorted_indices]
    sorted_categories = [categories[i] for i in sorted_indices]

    filtered_counts = sorted_counts[:top_n]
    filtered_labels = sorted_categories[:top_n]
    others_count = sum(sorted_counts[top_n:])
    
    if others_count > 0:
        filtered_counts.append(others_count)
        filtered_labels.append("Inne")
    
    return filtered_counts, filtered_labels

fig, axs = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("Procentowy udział cech według gatunku", fontsize=20)

def plot_genre_category_count_pie(ax, genre, title):
    categories = list(genre_category_counts[genre].sort_values(ascending=False)[1:].keys())
    counts = list(genre_category_counts[genre].sort_values(ascending=False)[1:])
    filtered_counts, filtered_labels = pie_data(counts, categories)
    colors = [value_color_dict.get(label, "#0097b8") for label in filtered_labels]
    ax.pie(filtered_counts, labels=filtered_labels, colors=colors, autopct="%1.1f%%")
    ax.set_title(title, size=16)

plot_genre_category_count_pie(axs[0,0], "R", "RPG")
plot_genre_category_count_pie(axs[0,1], "A", "Akcja")
plot_genre_category_count_pie(axs[1,0], "P", "Puzzle")
plot_genre_category_count_pie(axs[1,1], "S", "Strategia")

plt.tight_layout()
plt.show()

### Sentiment towards each aspect in aspect category

#### Feature: opinion, sentiment, catergory

In [ ]:
sentiment_aspect = {}
for genre, features in zip(df["genre"], df["feature"]):
    if genre not in sentiment_aspect:
        sentiment_aspect[genre] = {}
    
    for feature in features:
        if "category" in feature and "sentiment" in feature and "feature" in feature:
            category = feature["category"]
            sentiment = feature["sentiment"]
            feature = feature["feature"]
            if category not in sentiment_aspect[genre]:
                sentiment_aspect[genre][category] = {}
            if feature not in sentiment_aspect[genre][category]:
                sentiment_aspect[genre][category][feature] = []
            sentiment_aspect[genre][category][feature].append(sentiment)

#### Mean, median, standard deviation, variance, minimum, maximum

In [ ]:
data = []

for i,j in sentiment_aspect.items():
    for k, l in j.items():
        for m, n in l.items():
            data.append([i, k, m, len(n), np.mean(n), np.median(n), np.var(n), np.std(n), np.min(n), np.max(n)])

sentiment_aspect_df = pd.DataFrame(data, columns=["Genre", "Category", "Feature", "Count", "Mean", "Median", "Variance", "StdDev", "Min", "Max"])

sentiment_aspect_df

#### Frequency 

#### Most popular words in each category in genre

In [ ]:
sentiment_aspect_df_sort= sentiment_aspect_df.sort_values(by="Count", ascending=False)
top_10_common = sentiment_aspect_df_sort.groupby("Genre", group_keys=False).apply(lambda x: x.head(10))
top_10_common

#### higher variance vs sentiment

In [ ]:
plt.scatter(sentiment_aspect_df["Mean"], sentiment_aspect_df["Variance"], alpha = 0.7)
plt.title("Sentiment mean vs. Variance")
plt.xlabel("Sentiment mean")
plt.ylabel("Variance")
plt.show()

In [ ]:
corr_mean_variance = sentiment_aspect_df[["Mean", "Variance"]].corr().iloc[0, 1]
corr_mean_variance

### Sentiment towards category

In [ ]:
sentiment_aspect_category = {}
for genre, features in zip(df["genre"], df["feature"]):
    if genre not in sentiment_aspect_category:
        sentiment_aspect_category[genre] = {}
    
    for feature in features:
        if "category" in feature and "sentiment" in feature:
            category = feature["category"]
            sentiment = feature["sentiment"]
            if category not in sentiment_aspect_category[genre]:
                sentiment_aspect_category[genre][category] = []
            sentiment_aspect_category[genre][category].append(sentiment)

In [ ]:
sentiment_category_data = []

for i,j in sentiment_aspect_category.items():
    for k, l in j.items():
        sentiment_category_data.append([i, k, len(l), np.mean(l), np.median(l), np.var(l), np.std(l), np.min(l), np.max(l)])

sentiment_category_df = pd.DataFrame(sentiment_category_data, columns=["Genre", "Category", "Count", "Mean", "Median", "Variance", "StdDev", "Min", "Max"])

sentiment_category_df

In [ ]:
sentiment_category_pivot = sentiment_category_df.pivot("Genre", "Category", "Mean")
plt.figure(figsize=(12, 6))
sns.heatmap(sentiment_category_pivot, annot=True, cmap="coolwarm")
plt.title("Average Sentiment Scores by Category and Genre")
plt.xlabel("Category")
plt.ylabel("Genre")
plt.xticks(rotation=90)
plt.yticks()
plt.tight_layout()
plt.show()

In [ ]:
sentiment_category_pivot = sentiment_category_df.pivot(index="Genre", columns="Category", values="Mean").reset_index()

In [ ]:
df_plot = sentiment_category_pivot.melt(id_vars="Genre", var_name="Category", value_name="Sentiment")

plt.figure(figsize=(14, 8))
sns.barplot(x="Category", y="Sentiment", hue="Genre", data=df_plot, palette="magma")

plt.title("Sentiment Scores by Category and Genre")
plt.xlabel("Category")
plt.ylabel("Sentiment")
plt.xticks(rotation=90)
plt.legend(title="Genre")

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle("xyz", fontsize=20)

def boxplot_category(ax, genre, title):
    labels, data = [*zip(*sentiment_aspect_category[genre].items())]  
    labels, data = sentiment_aspect_category[genre].keys(), sentiment_aspect_category[genre].values()
    ax.boxplot(data)
    ax.set_xticklabels(labels)
    ax.tick_params(axis="x", rotation=90)
    ax.set_title(title)

boxplot_category(axs[0,0], "R", "RPG")
boxplot_category(axs[0,1], "A", "Action")
boxplot_category(axs[1,0], "P", "Puzzle")
boxplot_category(axs[1,1], "S", "Strategy")

plt.tight_layout()
plt.show()

#### Polarized categories

In [ ]:
sentiment_category_df.sort_values(by="StdDev", ascending=False).head(10)

## Game time and sentiment

In [ ]:
genre_category_time = {}
for genre, playtime, features in zip(df["genre"], df["playtime_at_review"], df["feature"]):
    if genre not in genre_category_time:
        genre_category_time[genre] = {}
        
    for feature in features:
        if "category" in feature and "sentiment" in feature:
            category = feature["category"]
            sentiment = feature["sentiment"]
            if category not in genre_category_time[genre]:
                genre_category_time[genre][category] = []

            genre_category_time[genre][category].append((sentiment, playtime))

In [ ]:
playtime_sentiment_correlation = {}
for genre, categories in genre_category_time.items():
    playtime_sentiment_correlation[genre] = {}
    for category, sentiments_playtimes in categories.items():
        sentiments = [sentiment for sentiment, _ in sentiments_playtimes]
        playtimes = [playtime for _, playtime in sentiments_playtimes]
        if sentiments and playtimes and len(sentiments) > 1:  
            correlation, _ = pearsonr(playtimes, sentiments)
            playtime_sentiment_correlation[genre][category] = correlation

In [ ]:
playtime_sentiment_correlation_df = pd.DataFrame(playtime_sentiment_correlation).T

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(playtime_sentiment_correlation_df, annot=True, cmap='PiYG')

plt.title("Correlation between Sentiment Scores and Playtime by Category and Genre")
plt.xlabel("Category")
plt.ylabel("Genre")
plt.xticks(rotation=90, ha="right")
plt.yticks()
plt.tight_layout()
plt.show()

## Adj for the word "game"

In [ ]:
genre_game_adj = {}
for genre, features in zip(df["genre"], df["feature"]):
    if genre not in genre_game_adj:
        genre_game_adj[genre] = []
        
    for feature in features:
        if feature["feature"] == "game" and "sentiment" in feature:
            opinion = feature["opinion"]
            sentiment = feature["sentiment"]
            genre_game_adj[genre].append((opinion, sentiment))

#### Number of positive, negative, and neutral sentiments

In [ ]:
def sentiment_analysis(opinions):
    positive, neutral, negative = 0, 0, 0
    for opinion, sentiment in opinions:
        if sentiment > 0:
            positive += 1
        elif sentiment < 0:
            negative += 1
        else:
            neutral += 1
    return positive, neutral, negative

sentiment_distribution = {genre: sentiment_analysis(opinions) for genre, opinions in genre_game_adj.items()}

In [ ]:
plt.figure(figsize=(10, 6))

genres = list(sentiment_distribution.keys())
positive = np.array([sentiment_distribution[genre][0] for genre in genres])
neutral = np.array([sentiment_distribution[genre][1] for genre in genres])
negative = np.array([sentiment_distribution[genre][2] for genre in genres])

plt.bar(genres, positive, color="#96d800", label="Positive")
plt.bar(genres, neutral, bottom=positive, color="#fce655", label="Neutral")
plt.bar(genres, negative, bottom=neutral + positive, color="#b20918", label="Negative")

plt.tick_params(axis="x", rotation=90)
plt.xlabel('Genre')
plt.ylabel('Sentiment score')
plt.title('Sentiment distribution for the word "game" by genre')
plt.legend()

plt.show()

#### Most frequent opinions

In [ ]:
common_opinions = {genre: Counter([opinion for opinion, _ in opinions]).most_common(5)
                   for genre, opinions in genre_game_adj.items()}

common_opinions

#### Opinion Polarity Analysis

In [ ]:
threshold = 0.5
polarized_opinions = { 'positive': {}, 'negative': {} }

for genre, opinions in genre_game_adj.items():
    polarized_opinions['positive'][genre] = []
    polarized_opinions['negative'][genre] = []

    used_opinions = set()

    for opinion, sentiment in opinions:
        if sentiment > threshold and opinion not in used_opinions:
            polarized_opinions['positive'][genre].append((opinion, sentiment))
            used_opinions.add(opinion) 
        elif sentiment < -threshold and opinion not in used_opinions:
            polarized_opinions['negative'][genre].append((opinion, sentiment))
            used_opinions.add(opinion) 

    polarized_opinions['positive'][genre] = sorted(polarized_opinions['positive'][genre], key=lambda x: x[1], reverse=True)[:5]
    polarized_opinions['negative'][genre] = sorted(polarized_opinions['negative'][genre], key=lambda x: x[1], reverse=False)[:5]

for genre in polarized_opinions['positive']:
    print(f"Genre: {genre}")
    print("Top 5 Positive Opinions:")
    for opinion, sentiment in polarized_opinions['positive'][genre]:
        print(f"Opinion: '{opinion}', Sentiment: {sentiment}")
        
    print("Top 5 Negative Opinions:")
    for opinion, sentiment in polarized_opinions['negative'][genre]:
        print(f"Opinion: '{opinion}', Sentiment: {sentiment}")

#### Wordcloud

In [ ]:
def plot_wordcloud(genre, opinions):
    opinions_text = ' '.join(opinion for opinion, _ in opinions)
    wordcloud = WordCloud(max_font_size = 220,
                max_words = 100,
                width = 1200,
                height = 800,
                background_color = "black",
                colormap='RdPu_r').generate(opinions_text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for Genre {genre}", fontsize=16)
    plt.show()

for genre, opinions in genre_game_adj.items():
    plot_wordcloud(genre, opinions)

## Overall opinion and sentiment towards certain aspects

In [ ]:
sentiment_voted_up = {}
for genre, votes, features in zip(df["genre"], df["voted_up"], df["feature"]):
    if genre not in sentiment_voted_up:
        sentiment_voted_up[genre] = {}
    
    for feature in features:
        if "category" in feature and "sentiment" in feature:
            category = feature["category"]
            sentiment = feature["sentiment"]
            if votes not in sentiment_voted_up[genre]:
                sentiment_voted_up[genre][votes] = {}
            if category not in sentiment_voted_up[genre][votes]:
                sentiment_voted_up[genre][votes][category] = []
            sentiment_voted_up[genre][votes][category].append(sentiment)

In [ ]:
plot_data = []
for genre, votes_data in sentiment_voted_up.items():
    for voted_up, categories in votes_data.items():
        for category, sentiments in categories.items():
            for sentiment in sentiments:
                plot_data.append({"Genre": genre, "Voted_up": voted_up, "Category": category, "Sentiment": sentiment})

df_plot = pd.DataFrame(plot_data)

def boxplot_category(genre, title):
    plt.figure(figsize=(14, 8))
    df_plot_genre = df_plot[df_plot["Genre"] == genre]
    sns.boxplot(x="Category", y="Sentiment", hue="Voted_up", data=df_plot_genre)
    plt.tick_params(axis="x", rotation=90)
    plt.title(title)
    plt.tight_layout()
    plt.show()

boxplot_category("R", "RPG")


In [ ]:
boxplot_category("A", "Action")

In [ ]:
boxplot_category("P", "Puzzle")

In [ ]:
boxplot_category("S", "Strategy")

In [ ]:
voted_up_data = []

for genre, genre_dict in sentiment_voted_up.items():
    for voted_up, voted_up_dict in genre_dict.items():
        for category, category_list in voted_up_dict.items():
            voted_up_data.append([genre, voted_up, category, len(category_list), np.mean(category_list),
                                  np.median(category_list), np.var(category_list), np.std(category_list), 
                                  np.min(category_list), np.max(category_list)])

voted_up_df = pd.DataFrame(voted_up_data, columns=["Genre", "Voted_up", "Category", "Count", "Mean", "Median", "Variance", "StdDev", "Min", "Max"])

voted_up_df

In [ ]:
overall_pos = voted_up_df[voted_up_df["Voted_up"] == True]

In [ ]:
overall_pos_pivot = overall_pos.pivot("Genre", "Category", "Mean")
plt.figure(figsize=(10, 6))
sns.heatmap(overall_pos_pivot, annot=True, cmap="coolwarm")
plt.title("Average Sentiment Scores by Category and Genre in Voted Up review")
plt.xlabel("Category")
plt.ylabel("Genre")
plt.xticks(rotation=90)
plt.yticks()
plt.tight_layout()
plt.show()

In [ ]:
overall_neg = voted_up_df[voted_up_df["Voted_up"] == False]

In [ ]:
overall_neg_pivot = overall_neg.pivot("Genre", "Category", "Mean")
plt.figure(figsize=(10, 6))
sns.heatmap(round(overall_neg_pivot,2), annot=True, cmap="coolwarm")
plt.title("Average Sentiment Scores by Category and Genre in Voted down review")
plt.xlabel("Category")
plt.ylabel("Genre")
plt.xticks(rotation=90)
plt.yticks()
plt.tight_layout()
plt.show()

In [ ]:
correlation_voted = df_plot.groupby(['Genre', 'Category']).apply(lambda x: x[['Voted_up', 'Sentiment']].corr().iloc[0, 1]).reset_index(name='Correlation')

In [ ]:
heatmap_voted = correlation_voted.pivot("Genre", "Category", "Correlation")

plt.figure(figsize=(10, 6))
sns.heatmap(round(heatmap_voted,2), annot=True, cmap="coolwarm")
plt.title('Correlation between Voted_up and Sentiment by Genre and Category')
plt.tight_layout()
plt.show()